# Ejercicio 2
 
Utiliza Apache Spark y un conjunto de datos público de vuelos, como el conjunto de datos de vuelos de 2015 proporcionado por la Oficina de Estadísticas de Transporte de EE. UU., para realizar las siguientes tareas: 
 
a)  Descarga los datos de vuelos en formato CSV desde la URL pública. 

b)  Carga los datos en un DataFrame de Spark. 

c)  Calcula la cantidad promedio de retrasos en la llegada de vuelos en un aeropuerto específico. 

d)  Encuentra las 10 rutas de vuelo más populares (pares de aeropuertos) en términos de la cantidad de vuelos.

## Importar Librerias

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.functions import avg, count, expr
import pandas as pd
from pyspark.sql.functions import col

## Crear la sesion

In [0]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Ejercicio 3") \
    .getOrCreate()

## Cargar los datos a un dataframe

In [0]:
# Funcion para detectar la moda en una lista
def find_mode_manual(data_list):
    if not data_list:
        return None

    counts = {}
    for item in data_list:
        counts[item] = counts.get(item, 0) + 1

    max_count = 0
    mode_value = None

    for item, count in counts.items():
        if count > max_count:
            max_count = count
            mode_value = item
        elif count == max_count and mode_value is not None:
            mode_value = None
    
    if max_count == 1:
      return None

    return mode_value

In [0]:
# Funcion para detectar el delimitador 
def detect_which_delimiter(file_path, sample_lines=5):
    try:
        file_content = dbutils.fs.head(file_path)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    lines = file_content.split('\n')
    if not lines:
        return None
    
    delimiters = [',', '\t', ';', '|', ' ']
    delimiter_counts = {delim: [] for delim in delimiters}
    
    for line in lines:
        if not line.strip():
            continue
        for delim in delimiters:
            delimiter_counts[delim].append(line.count(delim))
    
    detected_delimiter = None
    max_val = 0
    for delim in delimiters:
        val = find_mode_manual(delimiter_counts[delim])
        if max_val < val:
            max_val = val
            detected_delimiter = delim

    return detected_delimiter

In [0]:
# No hice esto porque no se podian leer ciertos archivos dado que tenían diferente delimitador
# df = spark.read\
#   .option("delimiter", ",")\
#   .csv("/FileStore/DataEjercicio2", inferSchema=True, header=True)
# df.show()
# Tambien no defini un esquema porque habian muchas columnas y a pesar que la mejor practica es hacerlo y podia definirlo, la verdad para este ejercicio no lo vi necesario dado que no es la vida real así que solo hice un infer schema
path = "/FileStore/DataEjercicio2"
dfs = []
for csv in dbutils.fs.ls(path):
    csv_path = csv.path.split(":")[1]
    delimiter = detect_which_delimiter(csv_path)
    df = spark.read\
        .option("delimiter", delimiter)\
        .csv(csv_path, inferSchema=True, header=True)
    dfs.append(df)

[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]
[Truncated to first 65536 bytes]


In [0]:
result_df = None
for df in dfs:
    if result_df == None:
        result_df = df
    else:
        result_df = result_df.union(df)
result_df.show()

+----+-------+-----+------------+-----------+--------------------+-----------------+---------------------+----------+--------+-----------------+-----------------+---------------------+---------------------+------+--------------------+----------------+-----------------+---------------+----------+---------------+-------------------+-------------------+----+--------------------+--------------+---------------+-------------+--------+------------+--------+---------+-------------+---------+---------------+------------+--------+---------+-------+------------+--------+---------+-------------+---------+---------------+------------+---------+-----------------+--------+----------------+-------------------+--------+-------+--------+--------------+-------------+-------------+---------+--------------+-------------------+--------------+---------------+-----------------+--------------------+----------------+-----------------------+-------------+------------+------------+---------------+----------------

## Calcula la cantidad promedio de retrasos en la llegada de vuelos en un aeropuerto específico.

In [0]:
result_df.where(result_df["DEST"] == "IDA")\
  .agg({"DEP_DELAY": "avg"})\
  .show()
  # Calcular para el aeropuerto de IDA

+-----------------+
|   avg(DEP_DELAY)|
+-----------------+
|3.659408381265407|
+-----------------+



In [0]:
result_df.groupBy("DEST")\
  .agg({"DEP_DELAY": "avg"})\
  .show()

+----+------------------+
|DEST|    avg(DEP_DELAY)|
+----+------------------+
| PSE|14.323383084577115|
| INL| 2.104669887278583|
| MSY| 9.757205945201576|
| PPG| 9.224137931034482|
| GEG| 7.821969696969697|
| SNA| 6.697584038813803|
| BUR| 7.452513143025598|
| GRB|  7.81866870696251|
| GTF|3.3939252336448598|
| IDA| 3.659408381265407|
| GRR|10.024398714943128|
| JLN|12.614285714285714|
| PSG| 6.455431754874652|
| EUG| 8.596414852752881|
| PVD|10.452219145901502|
| GSO|10.363520947947524|
| MYR| 9.561557549150601|
| OAK| 9.917056841552531|
| MSN| 8.044538160872248|
| FAR| 8.993307215148548|
+----+------------------+
only showing top 20 rows



## Encuentra las 10 rutas de vuelo más populares (pares de aeropuertos) en términos de la cantidad de vuelos.

In [0]:
result_df.groupBy("ORIGIN", "DEST")\
  .agg(count(expr("*")).alias("cantidad_de_vuelos"))\
  .sort("cantidad_de_vuelos",ascending=[False])\
  .limit(10)\
  .show()

+------+----+------------------+
|ORIGIN|DEST|cantidad_de_vuelos|
+------+----+------------------+
|   SFO| LAX|             15116|
|   LAX| SFO|             14799|
|   JFK| LAX|             13113|
|   LAX| JFK|             13106|
|   LAS| LAX|             10657|
|   LGA| ORD|             10560|
|   LAX| LAS|             10539|
|   ORD| LGA|             10492|
|   JFK| SFO|              9280|
|   SFO| JFK|              9279|
+------+----+------------------+



In [0]:
result_df.groupBy("ORIGIN", "DEST")\
  .agg(count(expr("1")).alias("cantidad_de_vuelos"))\
  .sort("cantidad_de_vuelos",ascending=[False])\
  .limit(10)\
  .show()

+------+----+------------------+
|ORIGIN|DEST|cantidad_de_vuelos|
+------+----+------------------+
|   SFO| LAX|             15116|
|   LAX| SFO|             14799|
|   JFK| LAX|             13113|
|   LAX| JFK|             13106|
|   LAS| LAX|             10657|
|   LGA| ORD|             10560|
|   LAX| LAS|             10539|
|   ORD| LGA|             10492|
|   JFK| SFO|              9280|
|   SFO| JFK|              9279|
+------+----+------------------+

